In [2]:
import pymysql.cursors
#import jsonify
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.ensemble import RandomForestRegressor

In [4]:
conn = pymysql.connect(
    host="softwaredb.ce0otalnccc9.eu-west-1.rds.amazonaws.com",user="soft",password="password",database="dublinbikes",
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor
)


In [5]:
availability_query = "SELECT number, time, available_bikes, available_bike_stands FROM availability"
#weather_query = "SELECT number, time, temperature, wind_speed, wind_direction, weather_code FROM weather"
weather_query = "SELECT * FROM weather"
# Load the query results into pandas dataframes
availability_df = pd.read_sql_query(availability_query, conn)
weather_df = pd.read_sql_query(weather_query, conn)

# Merge the dataframes on "time" and "number" columns



# Close the database connection
conn.close()

C:\Users\tfiro\AppData\Local\Temp\ipykernel_17600\2178617842.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  availability_df = pd.read_sql_query(availability_query, conn)
C:\Users\tfiro\AppData\Local\Temp\ipykernel_17600\2178617842.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  weather_df = pd.read_sql_query(weather_query, conn)


In [6]:
availability_df['time'] = availability_df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
weather_df['time'] = weather_df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
merged_df = pd.merge(availability_df, weather_df, on=["number", "time"])

In [7]:
merged_df.to_csv('merged_data.csv', index=False)

In [8]:
merged_df.tail()

,number,time,available_bikes,available_bike_stands,temperature,wind_speed,wind_direction,weather_code,precipitation_sum,rain_sum,precipitation_probability
1313870,507,2023-03-10 16:40:02,1,0,6,3,76,1,8.1,3.1,100
1313871,507,2023-03-10 16:45:03,1,0,6,3,76,1,8.1,3.1,100
1313872,507,2023-03-10 16:50:03,1,0,6,3,76,1,8.1,3.1,100
1313873,507,2023-03-10 16:55:03,1,0,6,3,76,1,8.1,3.1,100
1313874,507,2023-03-10 17:00:07,1,0,5,8,90,1,8.1,3.1,100


In [48]:
merged_df.isnull().sum()

number                       0
time                         0
available_bikes              0
available_bike_stands        0
temperature                  0
wind_speed                   0
wind_direction               0
weather_code                 0
precipitation_sum            0
rain_sum                     0
precipitation_probability    0
dtype: int64

In [49]:
merged_df.describe().T

,count,mean,std,min,25%,50%,75%,max
number,1313647.0,58.856302,35.967582,1.0,29.0,58.0,89.0,507.0
available_bikes,1313647.0,11.505513,9.383810,0.0,4.0,10.0,17.0,40.0
available_bike_stands,1313647.0,20.054286,10.870357,0.0,12.0,20.0,29.0,40.0
temperature,1313647.0,9.041181,3.253094,1.0,7.0,9.0,11.0,19.0
wind_speed,1313647.0,16.903415,8.588150,1.0,11.0,16.0,22.0,44.0
wind_direction,1313647.0,199.609700,75.920632,3.0,146.0,214.0,262.0,360.0
weather_code,1313647.0,16.149440,23.068453,0.0,2.0,3.0,51.0,73.0
precipitation_sum,1313647.0,3.347413,4.983085,0.0,0.2,1.9,3.5,30.2
rain_sum,1313647.0,2.559542,3.872297,0.0,0.0,0.6,3.4,20.2
precipitation_probability,1313647.0,74.436469,31.617998,0.0,58.0,87.0,100.0,100.0


In [50]:
print(merged_df.dtypes)


number                         int64
time                          object
available_bikes                int64
available_bike_stands          int64
temperature                    int64
wind_speed                     int64
wind_direction                 int64
weather_code                   int64
precipitation_sum            float64
rain_sum                     float64
precipitation_probability      int64
dtype: object


In [9]:
merged_df['time'] = merged_df['time'].astype('datetime64')
merged_df['number'] = merged_df['number'].astype('category')
merged_df['weather_code'] = merged_df['weather_code'].astype('category')

In [10]:
merged_df['hour'] = merged_df['time'].dt.hour
merged_df['minute'] = merged_df['time'].dt.minute
merged_df['month'] = merged_df['time'].dt.month
merged_df['day'] = merged_df['time'].dt.day


In [11]:
merged_df['hour'] = merged_df['hour'].astype('category')
merged_df['minute'] = merged_df['minute'].astype('category')
merged_df['month'] = merged_df['month'].astype('category')
merged_df['day'] = merged_df['day'].astype('category')

In [12]:
station_dfs = {}
for station in merged_df['number'].unique():
    station_dfs[station] = merged_df[merged_df['number'] == station]


In [55]:
filtered_df = merged_df[merged_df['number'] == 88]
filtered_df

,number,time,available_bikes,available_bike_stands,temperature,wind_speed,wind_direction,weather_code,precipitation_sum,rain_sum,precipitation_probability,hour,minute,month,day
975744,88,2023-04-16 00:09:54,15,15,16,5,82,2,7.3,4.0,90,0,9,4,16
975745,88,2023-04-16 12:27:09,12,18,16,5,82,2,7.3,4.0,90,12,27,4,16
975746,88,2023-04-16 23:55:03,6,24,12,3,360,3,0.0,0.0,13,23,55,4,16
975747,88,2023-04-17 00:00:06,6,24,12,5,113,2,0.0,0.0,13,0,0,4,17
975748,88,2023-04-17 00:05:03,6,24,12,5,113,2,0.0,0.0,29,0,5,4,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975942,88,2023-04-17 16:15:03,9,21,17,20,140,1,0.0,0.0,74,16,15,4,17
975943,88,2023-04-17 16:20:03,11,19,17,20,140,1,0.0,0.0,74,16,20,4,17
975944,88,2023-04-17 16:25:03,11,19,17,20,140,1,0.0,0.0,74,16,25,4,17
975945,88,2023-04-17 16:30:04,11,19,17,20,140,1,0.0,0.0,74,16,30,4,17


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd

# Assuming station_dfs is a dictionary containing data for each station

for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code','precipitation_sum','rain_sum','precipitation_probability']]
    y = station_df['available_bikes']

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    model = KNeighborsRegressor(n_neighbors=2)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    filename = f"model_dep{station}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(model, file, pickle.HIGHEST_PROTOCOL)

Station 1 R-squared: 0.9118103099858063
Station 2 R-squared: 0.8158324920324929
Station 3 R-squared: 0.9065736049888897
Station 4 R-squared: 0.8629092629298131
Station 5 R-squared: 0.8887397669192236
Station 6 R-squared: 0.9227745909696262
Station 7 R-squared: 0.8884710174244934
Station 8 R-squared: 0.8580100134707255
Station 9 R-squared: 0.845804489272372
Station 10 R-squared: 0.8460152659564215
Station 11 R-squared: 0.9407179246488608
Station 12 R-squared: 0.8977170144104885
Station 13 R-squared: 0.9361958220684469
Station 14 R-squared: 0.8408996963349795
Station 15 R-squared: 0.8205990171816719
Station 16 R-squared: 0.8950850057770566
Station 17 R-squared: 0.9050699548357583
Station 18 R-squared: 0.8774398533948783
Station 19 R-squared: 0.8693423125226745
Station 20 R-squared: 0.8983531061791878
Station 21 R-squared: 0.8586920673830601
Station 22 R-squared: 0.8936777979354424
Station 23 R-squared: 0.8865866152425242
Station 24 R-squared: 0.8299983435370369
Station 25 R-squared: 0.92

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd

# Assuming station_dfs is a dictionary containing data for each station

for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code','precipitation_sum','rain_sum','precipitation_probability']]
    y = station_df['available_bike_stands']

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    model = KNeighborsRegressor(n_neighbors=2)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    filename = f"model_arr{station}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(model, file, pickle.HIGHEST_PROTOCOL)

Station 1 R-squared: 0.9054055329909676
Station 2 R-squared: 0.8382245406769573
Station 3 R-squared: 0.9009472636894702
Station 4 R-squared: 0.8827480394387843
Station 5 R-squared: 0.8865130523316969
Station 6 R-squared: 0.9053629545632578
Station 7 R-squared: 0.8858747081539521
Station 8 R-squared: 0.8452494451747148
Station 9 R-squared: 0.8418498692112817
Station 10 R-squared: 0.8402041835219073
Station 11 R-squared: 0.9492257932235814
Station 12 R-squared: 0.9083111306097099
Station 13 R-squared: 0.919698129528037
Station 14 R-squared: 0.8645694048133156
Station 15 R-squared: 0.84783151618211
Station 16 R-squared: 0.8893114422701045
Station 17 R-squared: 0.9140432108656705
Station 18 R-squared: 0.8902292562221273
Station 19 R-squared: 0.8671320879199823
Station 20 R-squared: 0.9043826287197976
Station 21 R-squared: 0.8791191142690854
Station 22 R-squared: 0.9005930848312711
Station 23 R-squared: 0.8591887713407063
Station 24 R-squared: 0.8553310560253647
Station 25 R-squared: 0.9123

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd

# Assuming station_dfs is a dictionary containing data for each station

for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code','precipitation_sum','rain_sum','precipitation_probability']]
    y = station_df['available_bikes']

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    model = KNeighborsRegressor(n_neighbors=2)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    station_num = 1
    time_str = '2023-03-07 19:35:03'
    weather_code = 801
    temperature = 10
    wind_speed = 6
    wind_direction = 240
    precipitation_sum = 0
    rain_sum = 1
    precipitation_probability = 0
    time = pd.to_datetime(time_str)
    hour = time.hour
    day=time.day
    minute=time.minute
    month=time.month
    prediction_input = [[hour,day,minute,month, temperature, wind_speed, wind_direction, weather_code,precipitation_sum,rain_sum,precipitation_probability]]
    prediction = model.predict(prediction_input)
    
    print(f"Predicted number of available bikes for Station {station} at {time_str}: {prediction[0]:.2f}")


Station 1 R-squared: 0.9052214532029257
Predicted number of available bikes for Station 1 at 2023-03-07 19:35:03: 11.00
Station 2 R-squared: 0.8191751880983837
Predicted number of available bikes for Station 2 at 2023-03-07 19:35:03: 7.50
Station 3 R-squared: 0.8892803621256783
Predicted number of available bikes for Station 3 at 2023-03-07 19:35:03: 10.00
Station 4 R-squared: 0.881122780175411
Predicted number of available bikes for Station 4 at 2023-03-07 19:35:03: 6.00


C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


Station 5 R-squared: 0.8906335931208358
Predicted number of available bikes for Station 5 at 2023-03-07 19:35:03: 0.00
Station 6 R-squared: 0.9373764394786509
Predicted number of available bikes for Station 6 at 2023-03-07 19:35:03: 2.00
Station 7 R-squared: 0.8881361090736135
Predicted number of available bikes for Station 7 at 2023-03-07 19:35:03: 5.00
Station 8 R-squared: 0.8719915065022866
Predicted number of available bikes for Station 8 at 2023-03-07 19:35:03: 11.50


C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


Station 9 R-squared: 0.8480152078372953
Predicted number of available bikes for Station 9 at 2023-03-07 19:35:03: 4.00
Station 10 R-squared: 0.8682397775144426
Predicted number of available bikes for Station 10 at 2023-03-07 19:35:03: 12.00
Station 11 R-squared: 0.9177902655354775
Predicted number of available bikes for Station 11 at 2023-03-07 19:35:03: 13.00
Station 12 R-squared: 0.9041771061016164
Predicted number of available bikes for Station 12 at 2023-03-07 19:35:03: 1.50


...

C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


Station 113 R-squared: 0.9474445016710938
Predicted number of available bikes for Station 113 at 2023-03-07 19:35:03: 19.00
Station 114 R-squared: 0.8987539234860928
Predicted number of available bikes for Station 114 at 2023-03-07 19:35:03: 30.50
Station 115 R-squared: 0.9303282070154493
Predicted number of available bikes for Station 115 at 2023-03-07 19:35:03: 2.00
Station 116 R-squared: 0.8436211084556743
Predicted number of available bikes for Station 116 at 2023-03-07 19:35:03: 0.00


C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


Station 117 R-squared: 0.926564718521873
Predicted number of available bikes for Station 117 at 2023-03-07 19:35:03: 35.00
Station 507 R-squared: 1.0
Predicted number of available bikes for Station 507 at 2023-03-07 19:35:03: 1.00


C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


In [39]:
from sklearn.ensemble import RandomForestRegressor
for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code','precipitation_sum','rain_sum','precipitation_probability']]
    y = station_df['available_bikes']

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    station_num = 1
    time_str = '2023-03-07 19:35:03'
    weather_code = 801
    temperature = 10
    wind_speed = 6
    wind_direction = 240
    precipitation_sum = 0
    rain_sum = 1
    precipitation_probability = 0
    time = pd.to_datetime(time_str)
    hour = time.hour
    day=time.day
    minute=time.minute
    month=time.month
    prediction_input = [[hour,day,minute,month, temperature, wind_speed, wind_direction, weather_code,precipitation_sum,rain_sum,precipitation_probability]]
    prediction = model.predict(prediction_input)
    
    print(f"Predicted number of available bikes for Station {station} at {time_str}: {prediction[0]:.2f}")


Station 1 R-squared: 0.9916663172624414
Predicted number of available bikes for Station 1 at 2023-03-07 19:35:03: 5.78


C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Station 2 R-squared: 0.9730395907373097
Predicted number of available bikes for Station 2 at 2023-03-07 19:35:03: 7.19


C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Station 3 R-squared: 0.9842140977704964
Predicted number of available bikes for Station 3 at 2023-03-07 19:35:03: 2.47


...

C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Station 117 R-squared: 0.9972693246881204
Predicted number of available bikes for Station 117 at 2023-03-07 19:35:03: 4.68


C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Station 507 R-squared: 1.0
Predicted number of available bikes for Station 507 at 2023-03-07 19:35:03: 1.00


C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [36]:


for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code']]
    y = station_df['available_bikes']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    
    #Save the model to a pickle file
    filename = f"model_dep{station}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(model, file, pickle.HIGHEST_PROTOCOL)


Station 1 R-squared: 0.9867872253561613
Station 2 R-squared: 0.9721500591590089
Station 3 R-squared: 0.9838158092530607
Station 4 R-squared: 0.9831058150492503
Station 5 R-squared: 0.9875828103779551
Station 6 R-squared: 0.988740189925538
Station 7 R-squared: 0.9892700867302229
Station 8 R-squared: 0.9754291164342122
Station 9 R-squared: 0.9705847930205733
Station 10 R-squared: 0.9674723784202923
Station 11 R-squared: 0.9951843705420563
Station 12 R-squared: 0.9839927711386595
Station 13 R-squared: 0.9950233815615571
Station 14 R-squared: 0.9773688950252006
Station 15 R-squared: 0.9493062733103416
Station 16 R-squared: 0.9859909150384321
Station 17 R-squared: 0.9890665629651426
Station 18 R-squared: 0.9867007409857216
Station 19 R-squared: 0.9892702561202132
Station 20 R-squared: 0.9945588207377392
Station 21 R-squared: 0.9808905173589689
Station 22 R-squared: 0.9841405202397284
Station 23 R-squared: 0.9821346919795375
Station 24 R-squared: 0.9726158063026429
Station 25 R-squared: 0.99

In [37]:
import pickle
from sklearn.ensemble import RandomForestRegressor

for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code']]
    y = station_df['available_bike_stands']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    
    #Save the model to a pickle file
    filename = f"model_arr{station}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(model, file, pickle.HIGHEST_PROTOCOL)


Station 1 R-squared: 0.9867332969376081
Station 2 R-squared: 0.971878202423507
Station 3 R-squared: 0.9839359870308078
Station 4 R-squared: 0.9832413070936238
Station 5 R-squared: 0.9876522924825722
Station 6 R-squared: 0.9901489886411049
Station 7 R-squared: 0.9893950314610388
Station 8 R-squared: 0.9755600129615017
Station 9 R-squared: 0.9711323415525325
Station 10 R-squared: 0.9659479113944609
Station 11 R-squared: 0.9976162111793714
Station 12 R-squared: 0.984089841980373
Station 13 R-squared: 0.9947601938632339
Station 14 R-squared: 0.9771840738905736
Station 15 R-squared: 0.9490928000513994
Station 16 R-squared: 0.985468866020147
Station 17 R-squared: 0.9889829806871412
Station 18 R-squared: 0.9868202956869013
Station 19 R-squared: 0.9893861819655021
Station 20 R-squared: 0.994570589502016
Station 21 R-squared: 0.9805045380487941
Station 22 R-squared: 0.9840073220953182
Station 23 R-squared: 0.9728143057913528
Station 24 R-squared: 0.9730020202515992
Station 25 R-squared: 0.99469